In [2]:
from google.colab import drive
drive.mount('/content/drive/')
!pip install transformers==3

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
     |████████████████████████████████| 757kB 5.3MB/s 
     |████████████████████████████████| 3.0MB 7.1MB/s 
     |████████████████████████████████| 870kB 33.6MB/s 
     |████████████████████████████████| 1.2MB 34.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=9094907e71bc5a33ae24ac157efb15739fa74629b6ba4e21c80b5e89802bc8db
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [4]:
import os
import json
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
import re
import transformers
from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn

# import tweepy as tw
import pandas as pd
# consumer_key= 'ykjaALTDUsfi71F9SGpq0W92g'
# consumer_secret= 'GxpkG4gRoaz9FM162RuBeVTMHERqVpFMAV2oR6uw4CNuYWWiIV'
# access_token= '1336352130328788992-R6bj7Z6zDqkjA8zITZ9v6HTHFbTgp7'
# access_token_secret= 'Slfn5uyweJBshvA33ECxHHnygJ26QdEfF8oKUR0Wj92NX'
# auth = tw.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tw.API(auth, wait_on_rate_limit=True)
# # Define the search term and the date_since date as variables
# search_words = "#covid19"
# date_since = "2018-11-16"
# tweets_full = tw.Cursor(api.search,
#               q=search_words,
#               lang="en",
#               since=date_since,
#               tweet_mode='extended').items(5)

tweets=['''I want to chime in about the incident that happened in Bengaluru a few days ago.RESPECT ++You guys are providing great service,  i am a regular customer of zomato and i dont have any complain so far,,,only appreciation and gratitude! The way u are handling this case is also remarkableI demand gender equality for all men. Women says that they suffer even when they are treated with special care. It is the Man who is facing gender inequality today and this is the truth. 
This girl should get jail if the man comes out to be not guilty.
Keep the public updated.You are one of the corrupt person. Rs 30 is less as delivery fee for 3 Km & thats why you piggy back 3 orders in the same. Later onwards you will start pillon ride as well on the same order.Principle of natural justice needs to be followed. We need to know both side story.
Let's police do their work so that truth can come out.next business idea will be to start bike service. They will do piggybacking customer & food on the same route. Ultimate objective is to get money.''']

# for tweet in tweets_full:
#     if 'retweeted_status' in dir(tweet):
#         tweets.append(tweet.retweeted_status.full_text)
#         #rint(tweet.retweeted_status.full_text)
#     else:
#         tweets.append(tweet.full_text)
#         # print(tweet.full_text)
print(tweets)


    
device = "cuda" if torch.cuda.is_available() else "cpu"

bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


def drop_emoji(x):
    x= re.sub(r"http\S+", "", x)
    x=re.sub(r"[^\w\d\!\?\s]+",'',x)
    drop_list = []
    for i,_x in enumerate(x):
        if ord(_x) >= 128:
            drop_list.append(i)
    new_x = []
    for i,_x in enumerate(x):
        if i not in drop_list:
            new_x.append(_x)
    return ''.join(new_x)


# print(tweets)
tweets=list(map(drop_emoji,tweets))
#print(tweets)
tokens_test = tokenizer.batch_encode_plus(
    tweets,
    max_length = 100,
    pad_to_max_length=True,
    truncation=False,
    padding=True
)

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
# test_y = torch.tensor(test_labels)


for param in bert.parameters():
    param.requires_grad = False

class BERT_Arch(nn.Module):
    def __init__(self, bert):
        
        super(BERT_Arch, self).__init__()

        self.bert = bert 
        
        # dropout layer
        self.dropout = nn.Dropout(0.1)
        
        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)
        
        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512,2)

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model  
        _, cls_hs = self.bert(sent_id, attention_mask=mask)
        
        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)
        
        # apply softmax activation
        x = self.softmax(x)

        return x


model = BERT_Arch(bert)
path = '/content/drive/MyDrive/CAPSTONE PROJECT - PRAJNA SIRISHA SUKANYA/code/saved_weights.pt'
model.load_state_dict(torch.load(path),strict=False)

with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()


preds =list(np.argmax(preds, axis = 1))
print(preds)
for i in range(len(preds)):
    if(preds[i]==0):
        preds[i]="Non-rumour"
    else:
        preds[i]="Rumour"
# print(classification_report(test_y, preds))

for i in range(len(tweets)):
    print(i+1,tweets[i],"\n",preds[i])

["I want to chime in about the incident that happened in Bengaluru a few days ago.RESPECT ++You guys are providing great service,  i am a regular customer of zomato and i dont have any complain so far,,,only appreciation and gratitude! The way u are handling this case is also remarkableI demand gender equality for all men. Women says that they suffer even when they are treated with special care. It is the Man who is facing gender inequality today and this is the truth. \nThis girl should get jail if the man comes out to be not guilty.\nKeep the public updated.You are one of the corrupt person. Rs 30 is less as delivery fee for 3 Km & thats why you piggy back 3 orders in the same. Later onwards you will start pillon ride as well on the same order.Principle of natural justice needs to be followed. We need to know both side story.\nLet's police do their work so that truth can come out.next business idea will be to start bike service. They will do piggybacking customer & food on the same r